In [28]:
# import lib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import make_moons,make_classification,make_regression,make_circles
import sys
from mpl_toolkits.mplot3d import Axes3D 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA,KernelPCA
np.random.seed(23)
np.set_printoptions(precision=3,suppress=True)
plt.style.use('seaborn')

import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.examples.tutorials.mnist import input_data

def _sym_decorrelation(W):
    """ Symmetric decorrelation
    i.e. W <- (W * W.T) ^{-1/2} * W
    """
    s, u = np.linalg.eigh(np.dot(W, W.T))
    # u (resp. s) contains the eigenvectors (resp. square roots of
    # the eigenvalues) of W * W.T
    return np.dot(np.dot(u * (1./(np.sqrt(s+1e-8))+1e-8), u.T), W)

In [29]:
# import data
mnist = input_data.read_data_sets('../../Dataset/MNIST/', one_hot=True)
train_data,train_label,val_data,val_label,test_data,test_label = mnist.train.images,mnist.train.labels,mnist.validation.images,mnist.validation.labels,mnist.test.images, mnist.test.labels
print(train_data.shape,train_label.shape,train_data.max(),train_data.min())
print(val_data.shape,val_label.shape,val_data.max(),val_data.min())
print(test_data.shape,test_label.shape,test_data.max(),test_data.min())

Extracting ../../Dataset/MNIST/train-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/train-labels-idx1-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../../Dataset/MNIST/t10k-labels-idx1-ubyte.gz
(55000, 784) (55000, 10) 1.0 0.0
(5000, 784) (5000, 10) 1.0 0.0
(10000, 784) (10000, 10) 1.0 0.0


In [35]:
# start session 
sess = tf.InteractiveSession()


In [31]:
#  create class
def arctan(x):  return tf.arctan(x)
def tan(x)   :  return tf.tan(x)
def d_arctan(x):return 1/(1+x**2)
def d_tanh(x):  return 1/tf.cos(x)

class FNN():
    
    def __init__(self,input,output,act,d_act):
        self.w = tf.Variable(tf.random.normal())
        self.mw,self.mv = np.zeros_like()
        
        self.a = np.ones(output)
        self.b = np.ones(output)
        self.c = np.zeros(output)
        
        self.act  = act
        self.d_act= d_act

In [37]:
def tf_arctan(x): return tf.atan(x)
def tf_tan(x)   : return tf.tan(x)

print(tf_arctan(3).eval())

InvalidArgumentError: No OpKernel was registered to support Op 'Atan' with these attrs.  Registered devices: [CPU,GPU], Registered kernels:
  device='CPU'; T in [DT_FLOAT]
  device='CPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_DOUBLE]

	 [[node Atan_4 (defined at <ipython-input-37-ec6ee6a931c0>:1)  = Atan[T=DT_INT32](Atan_4/x)]]

Caused by op 'Atan_4', defined at:
  File "C:\ProgramData\Miniconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Miniconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Miniconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\P